In [ ]:
# Algoritmos de clasificación: 
# Random Forest: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
# Regresión logística: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html
# SVC: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

# Grid search: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html

##############################################
###### Experimento de Random Forest ##########
##############################################

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
import joblib

In [19]:
train_df = pd.read_csv('../EDA/train.csv')
test_df = pd.read_csv('../EDA/test.csv')
validation_df = pd.read_csv('../EDA/val.csv')

# Separate target column
train_X = train_df.drop(columns=['WL'])
train_Y = train_df['WL']

test_X = test_df.drop(columns=['WL'])
test_Y = test_df['WL']

validation_X = validation_df.drop(columns=['WL'])
validation_Y = validation_df['WL']

Se va a realizar un experimento donde se varian hiperparametros y se busca la mejor solucion

In [20]:
# Create a dictionary to store the results
results = []

# Define the hyperparameter values to test
n_estimators_values = [50, 100]
max_depth_values = [10, 20]
min_samples_split_values = [2, 5]

Se entrena el modelo con todos los posibles valores de hiperparametros y se selecciona el que tenga mejores métricas

In [ ]:
# Loop through all combinations of hyperparameters
for n_estimators in n_estimators_values:
    for max_depth in max_depth_values:
        for min_samples_split in min_samples_split_values:
            # Initialize the model with the current hyperparameters
            model = RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                random_state=42
            )
            
            # Train the model
            model.fit(train_X, train_Y)
            
            # Make predictions on the validation set
            validation_pred = model.predict(validation_X)
            
            # Calculate evaluation metrics -> matriz de regresión, score, classification report
            acc = accuracy_score(validation_Y, validation_pred)
            cm = confusion_matrix(validation_Y, validation_pred)
            report = classification_report(validation_Y, validation_pred, output_dict=True)
            
            # Store the results in the dictionary
            results.append({
                "n_estimators": n_estimators,
                "max_depth": max_depth,
                "min_samples_split": min_samples_split,
                "accuracy": acc
            })

In [ ]:
# Convert the results to a DataFrame for better visualization
results_df = pd.DataFrame(results)
results_df

,n_estimators,max_depth,min_samples_split,accuracy
0,50,10,2,0.822581
1,50,10,5,0.784946
2,50,20,2,0.827957
3,50,20,5,0.809140
4,100,10,2,0.811828
5,100,10,5,0.793011
6,100,20,2,0.819892
7,100,20,5,0.819892


In [23]:
# Select the best hyperparameters based on accuracy
best_params = results_df.loc[results_df['accuracy'].idxmax()]

In [24]:
# Train the final model with the best hyperparameters on the full training set
best_model = RandomForestClassifier(
    n_estimators=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']) if pd.notnull(best_params['max_depth']) else None,
    min_samples_split=int(best_params['min_samples_split']),
    random_state=42
)
best_model.fit(train_X, train_Y) 

RandomForestClassifier(max_depth=20, n_estimators=50, random_state=42)

In [ ]:
# Evaluate the final model on the test set
test_pred = best_model.predict(test_X)

print("\nEvaluación en conjunto de prueba:")
print("Accuracy:", accuracy_score(test_Y, test_pred))
print("Confusion Matrix:\n", confusion_matrix(test_Y, test_pred))
print("Classification Report:\n", classification_report(test_Y, test_pred))


📊 Evaluación en conjunto de prueba:
Accuracy: 0.8261802575107297
Confusion Matrix:
 [[197  44]
 [ 37 188]]
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.82      0.83       241
           1       0.81      0.84      0.82       225

    accuracy                           0.83       466
   macro avg       0.83      0.83      0.83       466
weighted avg       0.83      0.83      0.83       466



In [ ]:
# Como ejemplo: se tiene los datos de un equipo en de un partido y se debe predecir el WL

# Index(['WL', 'W', 'L', 'W_PCT', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
#        'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
#        'BLK', 'TOV', 'PF', 'PTS', 'year', 'month', 'day', 'team_analyzed',
#        'local_team', 'visitor_team'],
#       dtype='object')

# Predict a new game result
# Create a game
new_data = {
    'W': [47], 
    'L': [30], 
    'W_PCT': [0.61], 
    'MIN': [240], 
    'FGM': [46], 
    'FGA': [84], 
    'FG_PCT': [0.548], 
    'FG3M': [22], 
    'FG3A': [40],
    'FG3_PCT': [0.55], 
    'FTM': [12], 
    'FTA': [17], 
    'FT_PCT': [0.706], 
    'OREB': [8], 
    'DREB': [35], 
    'REB': [43], 
    'AST': [25], 
    'STL': [6],
    'BLK': [2], 
    'TOV': [15], 
    'PF': [15], 
    'PTS': [126], 
    'year': [2025], 
    'month': [4], 
    'day': [6], 
    'team_analyzed': [11],
    'local_team': [24],
    'visitor_team': [11]
}

# Convert the new data to a DataFrame
new_data_df = pd.DataFrame(new_data)
# Use the same preprocessing steps as the training data
predicted_result = best_model.predict(new_data_df)

# Mostrar el resultado de la predicción
print(f"Predicted Result: {predicted_result[0]}")

Predicted Result: 1


In [27]:
# Save the model to a file
joblib.dump(best_model, 'best_random_forest_model.pkl')
print("Model saved as 'best_random_forest_model.pkl'")

Model saved as 'best_random_forest_model.pkl'


In [28]:
# Load the model from the file
loaded_model = joblib.load('best_random_forest_model.pkl')

# Use the loaded model to make predictions
loaded_predicted_result = loaded_model.predict(new_data_df)
print(f"Predicted Result from loaded model: {loaded_predicted_result[0]}")

Predicted Result from loaded model: 1


Se entrenó un modelo de clasificación utilizando el algoritmo Random Forest, evaluando diferentes combinaciones de hiperparámetros como n_estimators, max_depth y min_samples_split. El mejor rendimiento obtenido en el conjunto de validación fue una precisión de 0.82.

Para comprobar la capacidad predictiva del modelo, se utilizó un conjunto de datos representativo de un partido real, cuyo resultado era conocido previamente. El modelo predijo correctamente que el equipo visitante sería el ganador, lo cual coincidió con el desenlace real del encuentro.

Estos resultados permiten concluir que el modelo entrenado con Random Forest presenta un buen nivel de precisión, lo que sugiere que es posible predecir el resultado de un partido de la NBA con un alto grado de confiabilidad a partir de las estadísticas previas del juego. Este modelo fue el que menor precisión tuvo de los 3 modelos que se entrenaron.

